In [0]:
# before running this cell one should setup YOLO via setup.sh

import os
import numpy as np
import cv2

import zipfile
from pprint import pprint
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
tf.enable_eager_execution()

from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset.utils import  box_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

from data_loader import Parser
from detection2d import Detector2D
from detection3d import Detector3D

import messages
from messages import Detection2d, Detection3d
from transformer import Transformer
from frustums import single_frustum_iterator
from training.pointnet import find_person_position

In [0]:
class frustum_FP_sampler(object):
  """Samples FP pedestrian detections via frustum pointnet"""

  def __init__(self, model_path, dataset_path, labled_mode_flag):
    """Initialize folder path and setup input parser."""
    self.dataset_path = dataset_path
    self.labled_mode_flag = labled_mode_flag
    self.fp_pedestrian = []

    self.GT_count = 0
    self.detector_2d = Detector2D()
    self.detector_3d = Detector3D(model_path)

    self.cmap = plt.get_cmap('jet')
    

  def run_detector(self, num_frames, plot_det=False, 
                   fp_min_pts_num_threshold=75, fp_intersection_ratio=1./2., dest_path=None, camera_id=2):
    """
    Performs FP detections on data from parser.
    Arguments:
      num_frames (int): set equals to -1 for full dataset parsing

      camera_id (int from 0 to 4): what camera should be used for 2D YOLO detection

      plot_det (bool): visualize detection (image + 2D-bbox + lidar points segmented as pedestrians)

      fp_min_pts_num_threshold (int): minimal number of points in frustum (that are classified as pedestrians) to consider it a valid classification
      (e.g. if there are only 10 points segmented as pedestrian its not a valid detection)

      fp_intersection_ratio (float from 0 to 1): minimal fraction of correctly segmented points in frustum to consider it a TP detection

      dest_path (NoneType or string): Where false positive detection must be saved as .npy binary files
    """

    assert type(num_frames) == int
    assert camera_id in {0,1,2,3,4}
    assert type(plot_det) == bool
    assert type(fp_min_pts_num_threshold) == int
    assert type(fp_intersection_ratio) == float and (0 < fp_intersection_ratio < 1)
    assert dest_path is None or type(dest_path) == str

    self.parser = Parser(self.dataset_path, self.labled_mode_flag, camera_id)

    max_frame_num = float('inf') if num_frames == -1 else num_frames
    
    if dest_path is not None and not os.path.exists(dest_path):
      os.mkdir(dest_path)

    self.current_iter_id = 0 
    self.frame_number = 0
    
    # DEBUG:
    self.test_frustum_points = []
    self.test_frustum_points_2 = []

    with self.detector_2d:
      for image, velo, calib, labels_ in self._labled_parser(self.parser):
        print("Frame number: {0}".format(self.frame_number))
      
        GT_pedestrians_box_3D, GT_pedestrians_box_2D = self._parse_labels_to_boxes(labels_)

        self.GT_count += len(GT_pedestrians_box_3D)

        velo = velo[:, :4][velo[:, -1] == -1] # only labeled zones

        boxes, scores, labels = self.detector_2d(image[:, :, :3])

        detections_2d = self._convert_yolo_2_detections(boxes, scores, labels)
     
        pts_detections_3d = self.detector_3d(image, velo, calib, detections_2d, camera_id, shift_pos=False)

        transformer = Transformer(calib, camera_id)

        if plot_det: # render GT bboxes and YOLO bboxes
          for gt in GT_pedestrians_box_2D:
            cv2.rectangle(image, (int(gt[0,0]), int(gt[1,1])), (int(gt[1,0]), int(gt[0,1])), (0, 0, 255, 255), 3)

          for d in detections_2d:
            cv2.rectangle(image, (int(d.x_min), int(d.y_min)), (int(d.x_max), int(d.y_max)), (0, 255, 0, 255), 3)
        
        axis_shuffle_matr = np.array([[0,-1,0,0], [0,0,-1,0], [1,0,0,0], [0,0,0,1]])
        vtc_tr_matrix = np.dot(axis_shuffle_matr, calib["Tr_velo_to_cam_{0}".format(camera_id)].reshape(4,4)) # Get calibration matrix for required camera
        
        # Inverse matrix (project from sensor frame to vehicle frame)  
        inv_vtc_transform_matrix = np.linalg.inv(vtc_tr_matrix)[:3, :3].T         

        print("NUMBER OF DETECTIONS: {0}".format(len(detections_2d)))

        for det3d in pts_detections_3d:
          if len(det3d) == 0:
            continue

          person_pts_vc = np.dot(det3d - vtc_tr_matrix[:3, 3], inv_vtc_transform_matrix)
          
          self.test_frustum_points_2.append((np.isclose(velo[:, :3][:, None], person_pts_vc)).all(-1).argmax(0))

          if len(person_pts_vc) < fp_min_pts_num_threshold: # not enough segmented points in frustum
            print("Not enough points in frustum. Skipped.")
            continue

          print(self.current_iter_id, len(GT_pedestrians_box_3D))

          if len(GT_pedestrians_box_3D) > 0:
            max_intersection_pts = np.max(
                                          box_utils.compute_num_points_in_box_3d(
                                                        tf.constant(person_pts_vc), 
                                                        tf.constant(GT_pedestrians_box_3D)).eval(session=self.detector_2d.session)
                                          )
            
          # True positive detection (=> skip to next detection)
          if max_intersection_pts > fp_intersection_ratio * len(person_pts_vc):
            print("True positive detection.")
            continue              

          print("FP detections of size {0} created: {1}/{2}".format(len(person_pts_vc), self.current_iter_id + 1, max_frame_num))

          self.current_iter_id += 1
          if dest_path:
            np.save(os.path.join(dest_path, "{0}.npy".format(self.current_iter_id).zfill(12)), person_pts_vc)

          self.test_frustum_points.append(person_pts_vc)
          # segmented points variation
          if plot_det:
            projected_points, mask = transformer.project_points(person_pts_vc[:, :3], image.shape,
                                                                return_only_valid=True)
            for p in projected_points:
              color = tuple(int(255 * x) for x in self.cmap(p[2] / 40.0)[:3])
              cv2.circle(image, (int(p[0]), int(p[1])), 10, [255, 0, 0], -1)

        if plot_det:
          pprint("GT_pedestrians: {0}".format(len(GT_pedestrians_box_3D))) 
          cv2.imshow(cv2.resize(image, (820, 820)))
        
        self.frame_number += 1

        if max_frame_num <= self.current_iter_id:
            break


  def get_fp_point_cloud_size(self):
    return list(map(len,  self.fp_pedestrian))


  def _parse_labels_to_boxes(self, labels, obj_type=["PEDESTRIAN", "CYCLIST", "VEHICLE"]):
    boxes_2D = []
    boxes_3D = []

    for gt in labels:
      if gt['type'] in obj_type:
        boxes_2D.append(gt['bbox'].reshape(2,2))
        boxes_3D.append(np.hstack([gt['location'], np.flip(gt['dimensions']), gt['rotation_y']]))

    return np.array(boxes_3D), np.array(boxes_2D)


  def _labled_parser(self, parser):
    if self.labled_mode_flag:
        for image, velo, calib, labels in parser.all:
            yield image, velo, calib, labels
    else:
        for image, velo, calib in parser.all:
            yield image, velo, calib, []


  def _convert_yolo_2_detections(self, boxes, scores, labels):
    detections_2d = []
    for box, score, label in zip(boxes, scores, labels):
        if label != 0:  # person
            continue
        d = Detection2d()
        d.x_min = box[0]
        d.y_min = box[1]
        d.x_max = box[2]
        d.y_max = box[3]
        d.label = messages.pedestrian
        d.score = score
        detections_2d.append(d)
    return detections_2d

In [0]:
MODEL_PATH = "./aid-submission/models/pedestrian"
WAYMO_PATH = "./Waymo_to_Kitti_adapter_train_0001/"
LABLED_MODE_FLAG = True

In [0]:
FP_sampler = frustum_FP_sampler(MODEL_PATH, WAYMO_PATH, LABLED_MODE_FLAG)

for cam_id in range(0, 5):
  dest_path = "/content/FP_detections_camera_{0}/".format(cam_id)
  FP_sampler.run_detector(1600, plot_det=False, fp_min_pts_num_threshold=20, dest_path=dest_path, camera_id=cam_id)